In [2]:
import pandas as pd
import glob
from googletrans import *

In [ ]:
!pip install googletrans==3.1.0a0

In [3]:
translator = Translator()

In [6]:
tamil_news = pd.read_csv('valid.csv')
hindi_news = pd.read_csv('train_hi.csv')
df_en = tamil_news.copy()
df_en.head()

#file_extension = ".csv"
#all_filenames = [i for i in glob.glob(f"*{file_extension}")]
#pd.concat(pd.read_csv(file)for file in all_filenames)

,text,label
0,விழிப்புணர்வு நிகழ்ச்சிகள் காரணமாக சிறிய நகரங்...,business
1,இறைவனை விட இறைவனின் நாமமே சிறந்தது என்பார்கள் ...,spirituality
2,தொடர்ச்சியாக காதல் படங்களில் நடித்தீர்கள். காத...,tamil-cinema
3,மெரிக்க சந்தையில் சிறந்த வரவேற்பைப் பெற்றுவிட்...,business
4,பெரிய பதவியில் இருப்பவர்கள் அறிமுகமாவார்கள். ச...,spirituality


In [7]:
translations = {}
for column in df_en.columns:
  # unique elements of the column
  unique_elements = df_en[column].unique()
  for element in unique_elements:
      # add translation to the dictionary
      translations[element] = translator.translate(element).text
    
print(translations)

{'விழிப்புணர்வு நிகழ்ச்சிகள் காரணமாக சிறிய நகரங்களில் இருந்து மியூச்சுவல் பண்ட்களில் முதலீடு செய்பவர்களின் எண்ணிக்கை உயர்ந்திருக்கிறது. மியூச்சுவல் பண்ட் நிறுவனங்கள் கையாளும் மொத்த சொத்தில் சிறிய நகரங்களின் பங்கு 18 சதவீதமாக உயர்ந்திருக்கிறது.': 'Due to awareness programs, the number of people investing in mutual funds from small towns has increased. The share of small towns has risen to 18 percent of the total assets handled by mutual fund companies.', 'இறைவனை விட இறைவனின் நாமமே சிறந்தது என்பார்கள் அருளாளர்கள். அதனால்தான் நாமசங்கீர்த்தனம் என்பதே இறைவனை உபாசனை செய்வதில் மிக உயர்ந்த மார்க்கமாக விளங்குகிறது. ராமா, கிருஷ்ணா என்னும் நாமங்களே, தீமைகள் நம்மை அண்டாமல் தடுத்திடும் வல்லமை உடையது என்றனர் சாய்ராமும் ஸ்ரீமாத்மிகாவும்.': "The benefactors say that God's name is better than God. That is why namasangeerthanam is the highest way of worshiping God. Sairam and Srimadmika said that the names of Rama and Krishna have the power to prevent us from suffering evil.", 'தொடர்ச்சியாக காதல் படங்கள

In [8]:
df_en.replace(translations, inplace = True)
df_en.head()

,text,label
0,"Due to awareness programs, the number of peopl...",business
1,The benefactors say that God's name is better ...,spirituality
2,You acted in a series of romantic films. Roman...,tamil-cinema
3,All industries believe that once they get a go...,business
4,Those who hold great positions will be introdu...,spirituality


In [13]:
#Sentiment Analysis
#Data prepocessing
from sklearn.feature_extraction.text import re

def clean_data(review):
    
    no_punc = re.sub(r'[^\w\s]', '', review)
    no_digits = ''.join([i for i in no_punc if not i.isdigit()])
    
    return(no_digits)

In [14]:
df_en['text'] = df_en['text'].apply(clean_data)
df_en['text'][0]


'Due to awareness programs the number of people investing in mutual funds from small towns has increased The share of small towns has risen to  percent of the total assets handled by mutual fund companies'

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False,
                        preprocessor=None)

X = tfidf.fit_transform(df_en['text'])

In [16]:
from sklearn.model_selection import train_test_split
y = df_en['Sentiment'] # target variable
X_train, X_test, y_train, y_test = train_test_split(X,y)


In [17]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train,y_train) # fit the model
preds = lr.predict(X_test) # make predictions

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(preds,y_test) #0.623
print(f"The model has an Accuracy of approximately {accuracy_score(preds,y_test)}")

The model has an Accuracy of approximately 0.6238805970149254


In [9]:
# Calculate sentiment score
import pandas as pd
import nltk
nltk.download('vader_lexicon')
nltk.download('SentiWordNet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()
#df = pd.read_csv('train_hi.csv')
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in df_en['text']]
df_en['sentiment_score'] = polarity
df_en.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Error loading SentiWordNet: Package 'SentiWordNet' not
[nltk_data]     found in index


,text,label,sentiment_score
0,"Due to awareness programs, the number of peopl...",business,0.65
1,The benefactors say that God's name is better ...,spirituality,-0.08
2,You acted in a series of romantic films. Roman...,tamil-cinema,0.66
3,All industries believe that once they get a go...,business,0.70
4,Those who hold great positions will be introdu...,spirituality,0.62


In [11]:
# Create the target variable
import numpy as np

def create_sentiment(sentiment_score):
    
    if sentiment_score <=0.1:
        return -1 # negative sentiment
    elif sentiment_score >=0.1:
        return 1 # positive sentiment
    else:
        return 0 # neutral sentiment

df_en['Sentiment'] = df_en['sentiment_score'].apply(create_sentiment)
df_en.head()

,text,label,sentiment_score,Sentiment
0,"Due to awareness programs, the number of peopl...",business,0.65,1
1,The benefactors say that God's name is better ...,spirituality,-0.08,-1
2,You acted in a series of romantic films. Roman...,tamil-cinema,0.66,1
3,All industries believe that once they get a go...,business,0.70,1
4,Those who hold great positions will be introdu...,spirituality,0.62,1


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [ ]:
# Text Translation
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    tokens = [token.text for token in doc]
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency
    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary = nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary = [word.text for word in summary]
    summary = ''.join(final_summary)
    return summary

In [ ]:
summarize(str(translations), per=0.005)

'Aamsa Yali will present to the devotees in special decorations including vehicles\', \'According to the central government  criminal cases have been registered against fake companies so far\': \'According to the central government  criminal cases have been registered against fake companies so far\', \'After the ceremony Kamal questioned Vishal to the press about the political controversy Vishal said\': \'After the ceremony Kamal questioned Vishal to the press about the political controversy Vishal said\', \' lamps lit and worshiped in the temple\': \'lamps lit and worshiped in the temple\', \'It has been decided to levy  percent additional tax cess on luxury vehicles and SUVs It is said that this will prevent companies manufacturing vehicles in this segment from undertaking future expansion plans\': \'It has been decided to levy  percent additional tax cess on luxury vehicles and SUVs It is said that this will prevent companies manufacturing vehicles in this segment from undertaking f